In [ ]:
from __future__ import print_function
import os
import warnings
warnings.filterwarnings('ignore')

import time
import torch
import shutil
import argparse
from m2det import build_net
import torch.utils.data as data
import torch.backends.cudnn as cudnn
from layers.functions import PriorBox
from data import detection_collate
from configs.CC import Config
from utils.core import *

args = []
config = 'configs/m2det512_resnet101.py'
dataset_prefix = 'VOC'
ngpu = 1
resume_net = None
resume_epoch = 0
tensorboard = False


print_info('----------------------------------------------------------------------\n'
           '|                       M2Det Training Program                       |\n'
           '----------------------------------------------------------------------',['yellow','bold'])

logger = set_logger(tensorboard)
global cfg
cfg = Config.fromfile(config)
net = build_net('train', 
                size = cfg.model.input_size, # Only 320, 512, 704 and 800 are supported
                config = cfg.model.m2det_config)
init_net(net, cfg, resume_net) # init the network with pretrained weights or resumed weights

if ngpu>1:
    net = torch.nn.DataParallel(net)
if cfg.train_cfg.cuda:
    net.cuda()
    cudnn.benchmark = True

optimizer = set_optimizer(net, cfg)
criterion = set_criterion(cfg)
priorbox = PriorBox(anchors(cfg))

with torch.no_grad():
    priors = priorbox.forward()
    if cfg.train_cfg.cuda:
        priors = priors.cuda()


net.train()
epoch = resume_epoch
print_info('===> Loading Dataset...',['yellow','bold'])
dataset = get_dataloader(cfg, dataset_prefix, 'train_sets')
epoch_size = len(dataset) // (cfg.train_cfg.per_batch_size * ngpu)
max_iter = cfg.train_cfg.step_lr[dataset_prefix][-1] * epoch_size
stepvalues = [_*epoch_size for _ in cfg.train_cfg.step_lr[dataset_prefix][:-1]]
print_info('===> Training M2Det on ' + dataset_prefix, ['yellow','bold'])
step_index = 0
if resume_epoch > 0:
    start_iter = resume_epoch * epoch_size
else:
    start_iter = 0
for iteration in range(start_iter, max_iter):
    if iteration % epoch_size == 0:
        batch_iterator = iter(data.DataLoader(dataset, 
                                              cfg.train_cfg.per_batch_size * ngpu, 
                                              shuffle=True, 
                                              num_workers=cfg.train_cfg.num_workers, 
                                              collate_fn=detection_collate))
        if epoch % cfg.model.save_eposhs == 0:
            save_checkpoint(net, cfg, final=False, datasetname = dataset_prefix, epoch=epoch)
        epoch += 1
    load_t0 = time.time()
    if iteration in stepvalues:
        step_index += 1
    lr = adjust_learning_rate(optimizer, cfg.train_cfg.gamma, epoch, step_index, iteration, epoch_size, cfg)
    images, targets = next(batch_iterator)
    if cfg.train_cfg.cuda:
        images = images.cuda()
        targets = [anno.cuda() for anno in targets]
    out = net(images)
    optimizer.zero_grad()
    loss_l, loss_c = criterion(out, priors, targets)
    loss = loss_l + loss_c
#     write_logger({'loc_loss':loss_l.item(),
#                   'conf_loss':loss_c.item(),
#                   'loss':loss.item()},logger,iteration,status=tensorboard)
    loss.backward()
    optimizer.step()
    load_t1 = time.time()
    print_train_log(iteration, cfg.train_cfg.print_epochs,
                        [time.ctime(),epoch,iteration%epoch_size,epoch_size,iteration,loss_l.item(),loss_c.item(),load_t1-load_t0,lr])
check_point_path = save_checkpoint(net, cfg, final=True, datasetname=dataset_prefix,epoch=-1)


----------------------------------------------------------------------
|                       M2Det Training Program                       |
----------------------------------------------------------------------
===> Constructing M2Det model
Initializing weights for [tums, reduce, up_reduce, leach, loc, conf]...
===> Loading Dataset...
===> Training M2Det on VOC
Time:Thu Jun 20 23:33:56 2019||Epoch:1||EpochIter:0/4187||Iter:0||Loss_L:3.0560||Loss_C:13.7343||Batch_Time:16.3733||LR:0.0000010
Time:Thu Jun 20 23:34:04 2019||Epoch:1||EpochIter:10/4187||Iter:10||Loss_L:3.6664||Loss_C:13.7101||Batch_Time:0.7808||LR:0.0000029
Time:Thu Jun 20 23:34:12 2019||Epoch:1||EpochIter:20/4187||Iter:20||Loss_L:3.6959||Loss_C:13.5767||Batch_Time:0.7844||LR:0.0000048
Time:Thu Jun 20 23:34:20 2019||Epoch:1||EpochIter:30/4187||Iter:30||Loss_L:3.0639||Loss_C:12.9943||Batch_Time:0.7876||LR:0.0000067
Time:Thu Jun 20 23:34:28 2019||Epoch:1||EpochIter:40/4187||Iter:40||Loss_L:3.3731||Loss_C:12.7597||Batch_Time:0

Time:Thu Jun 20 23:41:56 2019||Epoch:1||EpochIter:550/4187||Iter:550||Loss_L:3.3505||Loss_C:10.9186||Batch_Time:0.8742||LR:0.0001061
Time:Thu Jun 20 23:42:05 2019||Epoch:1||EpochIter:560/4187||Iter:560||Loss_L:2.8166||Loss_C:12.1068||Batch_Time:0.8740||LR:0.0001080
Time:Thu Jun 20 23:42:14 2019||Epoch:1||EpochIter:570/4187||Iter:570||Loss_L:3.0078||Loss_C:11.8359||Batch_Time:0.8730||LR:0.0001099
Time:Thu Jun 20 23:42:23 2019||Epoch:1||EpochIter:580/4187||Iter:580||Loss_L:2.7464||Loss_C:11.1760||Batch_Time:0.8744||LR:0.0001118
Time:Thu Jun 20 23:42:31 2019||Epoch:1||EpochIter:590/4187||Iter:590||Loss_L:2.6618||Loss_C:11.2519||Batch_Time:0.8740||LR:0.0001137
Time:Thu Jun 20 23:42:40 2019||Epoch:1||EpochIter:600/4187||Iter:600||Loss_L:2.5558||Loss_C:11.5901||Batch_Time:0.8748||LR:0.0001156
Time:Thu Jun 20 23:42:49 2019||Epoch:1||EpochIter:610/4187||Iter:610||Loss_L:4.1370||Loss_C:11.1566||Batch_Time:0.8734||LR:0.0001175
Time:Thu Jun 20 23:42:58 2019||Epoch:1||EpochIter:620/4187||Iter:620|

Time:Thu Jun 20 23:50:29 2019||Epoch:1||EpochIter:1130/4187||Iter:1130||Loss_L:3.3908||Loss_C:5.6019||Batch_Time:0.8735||LR:0.0002169
Time:Thu Jun 20 23:50:37 2019||Epoch:1||EpochIter:1140/4187||Iter:1140||Loss_L:3.4466||Loss_C:4.6719||Batch_Time:0.8742||LR:0.0002188
Time:Thu Jun 20 23:50:46 2019||Epoch:1||EpochIter:1150/4187||Iter:1150||Loss_L:3.4429||Loss_C:5.3507||Batch_Time:0.8725||LR:0.0002207
Time:Thu Jun 20 23:50:55 2019||Epoch:1||EpochIter:1160/4187||Iter:1160||Loss_L:2.5229||Loss_C:4.3813||Batch_Time:0.8730||LR:0.0002226
Time:Thu Jun 20 23:51:04 2019||Epoch:1||EpochIter:1170/4187||Iter:1170||Loss_L:2.5853||Loss_C:4.1007||Batch_Time:0.8732||LR:0.0002245
Time:Thu Jun 20 23:51:13 2019||Epoch:1||EpochIter:1180/4187||Iter:1180||Loss_L:3.2070||Loss_C:3.8009||Batch_Time:0.8724||LR:0.0002264
Time:Thu Jun 20 23:51:22 2019||Epoch:1||EpochIter:1190/4187||Iter:1190||Loss_L:3.6476||Loss_C:5.2690||Batch_Time:0.8716||LR:0.0002283
Time:Thu Jun 20 23:51:30 2019||Epoch:1||EpochIter:1200/4187||I

Time:Thu Jun 20 23:59:01 2019||Epoch:1||EpochIter:1710/4187||Iter:1710||Loss_L:3.0080||Loss_C:3.0586||Batch_Time:0.8738||LR:0.0003276
Time:Thu Jun 20 23:59:10 2019||Epoch:1||EpochIter:1720/4187||Iter:1720||Loss_L:2.7027||Loss_C:3.4208||Batch_Time:0.8749||LR:0.0003296
Time:Thu Jun 20 23:59:19 2019||Epoch:1||EpochIter:1730/4187||Iter:1730||Loss_L:2.6190||Loss_C:2.8339||Batch_Time:0.8751||LR:0.0003315
Time:Thu Jun 20 23:59:28 2019||Epoch:1||EpochIter:1740/4187||Iter:1740||Loss_L:3.1398||Loss_C:4.2215||Batch_Time:0.8707||LR:0.0003334
Time:Thu Jun 20 23:59:37 2019||Epoch:1||EpochIter:1750/4187||Iter:1750||Loss_L:3.0787||Loss_C:3.2352||Batch_Time:0.8741||LR:0.0003353
Time:Thu Jun 20 23:59:45 2019||Epoch:1||EpochIter:1760/4187||Iter:1760||Loss_L:2.4181||Loss_C:3.4468||Batch_Time:0.8743||LR:0.0003372
Time:Thu Jun 20 23:59:54 2019||Epoch:1||EpochIter:1770/4187||Iter:1770||Loss_L:3.0551||Loss_C:3.0689||Batch_Time:0.8727||LR:0.0003391
Time:Fri Jun 21 00:00:03 2019||Epoch:1||EpochIter:1780/4187||I

Time:Fri Jun 21 00:07:34 2019||Epoch:1||EpochIter:2290/4187||Iter:2290||Loss_L:2.1438||Loss_C:3.4608||Batch_Time:0.8744||LR:0.0004384
Time:Fri Jun 21 00:07:43 2019||Epoch:1||EpochIter:2300/4187||Iter:2300||Loss_L:2.2528||Loss_C:2.9418||Batch_Time:0.8766||LR:0.0004403
Time:Fri Jun 21 00:07:52 2019||Epoch:1||EpochIter:2310/4187||Iter:2310||Loss_L:2.4312||Loss_C:3.6815||Batch_Time:0.8747||LR:0.0004423
Time:Fri Jun 21 00:08:01 2019||Epoch:1||EpochIter:2320/4187||Iter:2320||Loss_L:2.9591||Loss_C:3.1650||Batch_Time:0.8741||LR:0.0004442
Time:Fri Jun 21 00:08:10 2019||Epoch:1||EpochIter:2330/4187||Iter:2330||Loss_L:2.4474||Loss_C:3.1674||Batch_Time:0.8732||LR:0.0004461
Time:Fri Jun 21 00:08:18 2019||Epoch:1||EpochIter:2340/4187||Iter:2340||Loss_L:2.5187||Loss_C:3.7047||Batch_Time:0.8742||LR:0.0004480
Time:Fri Jun 21 00:08:27 2019||Epoch:1||EpochIter:2350/4187||Iter:2350||Loss_L:2.5193||Loss_C:3.3579||Batch_Time:0.8742||LR:0.0004499
Time:Fri Jun 21 00:08:36 2019||Epoch:1||EpochIter:2360/4187||I

...

Time:Fri Jun 21 14:10:28 2019||Epoch:15||EpochIter:952/4187||Iter:59570||Loss_L:1.6860||Loss_C:0.6653||Batch_Time:0.8740||LR:0.0040000
Time:Fri Jun 21 14:10:37 2019||Epoch:15||EpochIter:962/4187||Iter:59580||Loss_L:0.3561||Loss_C:0.0673||Batch_Time:0.8724||LR:0.0040000
Time:Fri Jun 21 14:10:46 2019||Epoch:15||EpochIter:972/4187||Iter:59590||Loss_L:1.3495||Loss_C:0.0748||Batch_Time:0.8719||LR:0.0040000
Time:Fri Jun 21 14:10:54 2019||Epoch:15||EpochIter:982/4187||Iter:59600||Loss_L:0.6475||Loss_C:0.2025||Batch_Time:0.8722||LR:0.0040000
Time:Fri Jun 21 14:11:03 2019||Epoch:15||EpochIter:992/4187||Iter:59610||Loss_L:1.4659||Loss_C:0.1011||Batch_Time:0.8727||LR:0.0040000
Time:Fri Jun 21 14:11:12 2019||Epoch:15||EpochIter:1002/4187||Iter:59620||Loss_L:0.5113||Loss_C:0.0363||Batch_Time:0.8732||LR:0.0040000
Time:Fri Jun 21 14:11:21 2019||Epoch:15||EpochIter:1012/4187||Iter:59630||Loss_L:0.8580||Loss_C:0.1049||Batch_Time:0.8720||LR:0.0040000
Time:Fri Jun 21 14:11:30 2019||Epoch:15||EpochIter:10

Time:Fri Jun 21 14:18:51 2019||Epoch:15||EpochIter:1522/4187||Iter:60140||Loss_L:0.6347||Loss_C:0.1044||Batch_Time:0.8724||LR:0.0040000
Time:Fri Jun 21 14:19:00 2019||Epoch:15||EpochIter:1532/4187||Iter:60150||Loss_L:1.4066||Loss_C:0.1502||Batch_Time:0.8727||LR:0.0040000
Time:Fri Jun 21 14:19:09 2019||Epoch:15||EpochIter:1542/4187||Iter:60160||Loss_L:0.7699||Loss_C:0.3383||Batch_Time:0.8719||LR:0.0040000
Time:Fri Jun 21 14:19:18 2019||Epoch:15||EpochIter:1552/4187||Iter:60170||Loss_L:0.8775||Loss_C:0.0245||Batch_Time:0.8727||LR:0.0040000
Time:Fri Jun 21 14:19:26 2019||Epoch:15||EpochIter:1562/4187||Iter:60180||Loss_L:0.7066||Loss_C:0.0585||Batch_Time:0.8731||LR:0.0040000
Time:Fri Jun 21 14:19:35 2019||Epoch:15||EpochIter:1572/4187||Iter:60190||Loss_L:0.7993||Loss_C:0.0900||Batch_Time:0.8735||LR:0.0040000
Time:Fri Jun 21 14:19:44 2019||Epoch:15||EpochIter:1582/4187||Iter:60200||Loss_L:1.6291||Loss_C:0.0800||Batch_Time:0.8731||LR:0.0040000
Time:Fri Jun 21 14:19:53 2019||Epoch:15||EpochIt

Time:Fri Jun 21 14:27:14 2019||Epoch:15||EpochIter:2092/4187||Iter:60710||Loss_L:0.9444||Loss_C:0.1369||Batch_Time:0.8722||LR:0.0040000
Time:Fri Jun 21 14:27:23 2019||Epoch:15||EpochIter:2102/4187||Iter:60720||Loss_L:0.7461||Loss_C:0.0605||Batch_Time:0.8727||LR:0.0040000
Time:Fri Jun 21 14:27:32 2019||Epoch:15||EpochIter:2112/4187||Iter:60730||Loss_L:1.2293||Loss_C:0.2159||Batch_Time:0.8722||LR:0.0040000
Time:Fri Jun 21 14:27:41 2019||Epoch:15||EpochIter:2122/4187||Iter:60740||Loss_L:0.8709||Loss_C:0.0781||Batch_Time:0.8710||LR:0.0040000
Time:Fri Jun 21 14:27:50 2019||Epoch:15||EpochIter:2132/4187||Iter:60750||Loss_L:0.5835||Loss_C:0.0520||Batch_Time:0.8720||LR:0.0040000
Time:Fri Jun 21 14:27:58 2019||Epoch:15||EpochIter:2142/4187||Iter:60760||Loss_L:0.5673||Loss_C:0.0842||Batch_Time:0.8723||LR:0.0040000
Time:Fri Jun 21 14:28:07 2019||Epoch:15||EpochIter:2152/4187||Iter:60770||Loss_L:1.3600||Loss_C:0.0948||Batch_Time:0.8711||LR:0.0040000
Time:Fri Jun 21 14:28:16 2019||Epoch:15||EpochIt

Time:Fri Jun 21 14:35:37 2019||Epoch:15||EpochIter:2662/4187||Iter:61280||Loss_L:0.7148||Loss_C:0.0250||Batch_Time:0.8716||LR:0.0040000
Time:Fri Jun 21 14:35:46 2019||Epoch:15||EpochIter:2672/4187||Iter:61290||Loss_L:0.6385||Loss_C:0.0567||Batch_Time:0.8728||LR:0.0040000
Time:Fri Jun 21 14:35:55 2019||Epoch:15||EpochIter:2682/4187||Iter:61300||Loss_L:0.9733||Loss_C:0.0854||Batch_Time:0.8740||LR:0.0040000
Time:Fri Jun 21 14:36:04 2019||Epoch:15||EpochIter:2692/4187||Iter:61310||Loss_L:0.5783||Loss_C:0.0607||Batch_Time:0.8725||LR:0.0040000
Time:Fri Jun 21 14:36:13 2019||Epoch:15||EpochIter:2702/4187||Iter:61320||Loss_L:0.7919||Loss_C:0.0513||Batch_Time:0.8732||LR:0.0040000
Time:Fri Jun 21 14:36:22 2019||Epoch:15||EpochIter:2712/4187||Iter:61330||Loss_L:0.8872||Loss_C:0.0493||Batch_Time:0.8739||LR:0.0040000
Time:Fri Jun 21 14:36:30 2019||Epoch:15||EpochIter:2722/4187||Iter:61340||Loss_L:1.2187||Loss_C:0.0771||Batch_Time:0.8725||LR:0.0040000
Time:Fri Jun 21 14:36:39 2019||Epoch:15||EpochIt

Time:Fri Jun 21 14:44:01 2019||Epoch:15||EpochIter:3232/4187||Iter:61850||Loss_L:1.0337||Loss_C:0.0228||Batch_Time:0.8728||LR:0.0040000
Time:Fri Jun 21 14:44:09 2019||Epoch:15||EpochIter:3242/4187||Iter:61860||Loss_L:0.5220||Loss_C:0.1192||Batch_Time:0.8723||LR:0.0040000
Time:Fri Jun 21 14:44:18 2019||Epoch:15||EpochIter:3252/4187||Iter:61870||Loss_L:0.9370||Loss_C:0.0304||Batch_Time:0.8729||LR:0.0040000
Time:Fri Jun 21 14:44:27 2019||Epoch:15||EpochIter:3262/4187||Iter:61880||Loss_L:0.5657||Loss_C:0.0158||Batch_Time:0.8711||LR:0.0040000
Time:Fri Jun 21 14:44:36 2019||Epoch:15||EpochIter:3272/4187||Iter:61890||Loss_L:1.0982||Loss_C:0.0436||Batch_Time:0.8738||LR:0.0040000
Time:Fri Jun 21 14:44:45 2019||Epoch:15||EpochIter:3282/4187||Iter:61900||Loss_L:0.6444||Loss_C:0.0456||Batch_Time:0.8725||LR:0.0040000
Time:Fri Jun 21 14:44:54 2019||Epoch:15||EpochIter:3292/4187||Iter:61910||Loss_L:1.0721||Loss_C:0.1724||Batch_Time:0.8754||LR:0.0040000
Time:Fri Jun 21 14:45:02 2019||Epoch:15||EpochIt

In [ ]:
print('done')

In [ ]:
check_point_path